In [32]:
! /opt/nest/bin/nest-server start -d

In [33]:
import ray
ray.init()

2022-12-15 15:43:49,839	WARNING services.py:1732 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=5.03gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2022-12-15 15:43:49,991	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.6
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8265


In [55]:
from NESTServerClient import NESTServerClient  # , requests  # encode,
import requests
from werkzeug.exceptions import BadRequest


def encode(response):
    if response.ok:
        return response.json()
    elif response.status_code == 400:
        raise BadRequest(response.text)
 

@ray.remote
def method(call, url, headers, *args, **kwargs):
    kwargs.update({'args': args})
    response = requests.post(url + 'api/' + call, json=kwargs, headers=headers)
    return encode(response)
     
    
@ray.remote
def exec_script(url, headers, source, return_vars=None):
    params = {
        'source': source,
        'return': return_vars,
    }
    response = requests.post(url + 'exec', json=params, headers=headers)
    return encode(response)


def from_file(url, headers, filename, return_vars=None):
    with open(filename, 'r') as f:
        lines = f.readlines()
        script = ''.join(lines)
        print('Execute script code of {}'.format(filename))
        print('Return variables: {}'.format(return_vars))
        print(20 * '-')
        print(script)
        print(20 * '-')
    return exec_script.remote(url, headers, script, return_vars)


class RayNESTServerClient(NESTServerClient):

    obj_refs = list()
    
    def __init__(self, host='localhost', port=52425):
        super(RayNESTServerClient, self).__init__(host=host, port=port)
        self.obj_ref = list()
        
    def run_async_command(self, call, *args, **kwargs):
        ray_method = lambda *args, **kwargs: method.remote(call, self.url, self.headers, *args, **kwargs)
        obj_ref = ray_method(*args, **kwargs)
        self.obj_refs.append(obj_ref)
        return obj_ref

    def __getattr__(self, call):
        if "async" in call:
            call = call.replace("async", "")
            return lambda *args, **kwargs: self.run_async_command(call, *args, **kwargs)
        else:
            return super(RayNESTServerClient, self).__getattr__(call)
       
    def exec_script(self, source, return_vars=None, async_flag=False):
        if async_flag:
            obj_ref = exec_script.remote(self.url, self.headers, source, return_vars=return_vars)
            self.obj_refs.append(obj_ref)
            return obj_ref
        else:
            return super(RayNESTServerClient, self).exec_script(source, return_vars)
        
    def from_file(self, filename, return_vars=None):
        if async_flag:
            obj_ref = from_file.remote(self.url, self.headers, filename, return_vars=return_vars)
            self.obj_refs.append(obj_ref)
            return obj_ref
        else:
            return super(RayNESTServerClient, self).from_file(filename, return_vars)

        
    def __getstate__(self):
        return {"obj_refs": list(self.obj_refs)}
    
    def __setstate__(self, d):
        self.obj_refs = d.get("obj_refs", list())
    
        

In [42]:
nsc = RayNESTServerClient(port=5000)


In [43]:
nsc.GetKernelStatus()

{'T_max': 1152921504606846.8,
 'T_min': -1152921504606846.8,
 'adaptive_spike_buffers': True,
 'adaptive_target_buffers': True,
 'biological_time': 0.0,
 'buffer_size_spike_data': 2,
 'buffer_size_target_data': 2,
 'connection_rules': ['one_to_one',
  'all_to_all',
  'fixed_indegree',
  'fixed_outdegree',
  'pairwise_bernoulli',
  'symmetric_pairwise_bernoulli',
  'fixed_total_number'],
 'data_path': '',
 'data_prefix': '',
 'dict_miss_is_error': True,
 'growth_curves': ['gaussian', 'linear', 'sigmoid'],
 'growth_factor_buffer_spike_data': 1.5,
 'growth_factor_buffer_target_data': 1.5,
 'keep_source_table': True,
 'local_num_threads': 1,
 'local_spike_counter': 0,
 'max_buffer_size_spike_data': 8388608,
 'max_buffer_size_target_data': 16777216,
 'max_delay': 0.1,
 'max_num_syn_models': 512,
 'max_update_time': -inf,
 'min_delay': 0.1,
 'min_update_time': inf,
 'ms_per_tic': 0.001,
 'network_size': 0,
 'node_models': ['weight_recorder',
  'gauss_rate_ipn',
  'lin_rate_ipn',
  'sigmoid_r

In [44]:
ray.get(nsc.asyncGetKernelStatus())

{'T_max': 1152921504606846.8,
 'T_min': -1152921504606846.8,
 'adaptive_spike_buffers': True,
 'adaptive_target_buffers': True,
 'biological_time': 0.0,
 'buffer_size_spike_data': 2,
 'buffer_size_target_data': 2,
 'connection_rules': ['one_to_one',
  'all_to_all',
  'fixed_indegree',
  'fixed_outdegree',
  'pairwise_bernoulli',
  'symmetric_pairwise_bernoulli',
  'fixed_total_number'],
 'data_path': '',
 'data_prefix': '',
 'dict_miss_is_error': True,
 'growth_curves': ['gaussian', 'linear', 'sigmoid'],
 'growth_factor_buffer_spike_data': 1.5,
 'growth_factor_buffer_target_data': 1.5,
 'keep_source_table': True,
 'local_num_threads': 1,
 'local_spike_counter': 0,
 'max_buffer_size_spike_data': 8388608,
 'max_buffer_size_target_data': 16777216,
 'max_delay': 0.1,
 'max_num_syn_models': 512,
 'max_update_time': -inf,
 'min_delay': 0.1,
 'min_update_time': inf,
 'ms_per_tic': 0.001,
 'network_size': 0,
 'node_models': ['weight_recorder',
  'gauss_rate_ipn',
  'lin_rate_ipn',
  'sigmoid_r

In [45]:
ray.wait(nsc.obj_refs)

([ObjectRef(c54e76759b2a0c10ffffffffffffffffffffffff0100000001000000)], [])

In [46]:
# Create nodes
params = {"rate": 6500.}
pg = ray.get(nsc.asyncCreate("poisson_generator", 1, params))
neurons = ray.get(nsc.asyncCreate("iaf_psc_alpha", 1000))
sr = ray.get(nsc.asyncCreate("spike_recorder"))

# Connect nodes
ray.get(nsc.asyncConnect(pg, neurons, syn_spec={'weight': 10.}))
ray.get(nsc.asyncConnect(neurons[::10], sr))


In [47]:

# Simulate
nsc.asyncSimulate(1000.0)



ObjectRef(465c0fb8d6cb3cdcffffffffffffffffffffffff0100000001000000)

In [48]:
# Get events
n_events = ray.get(nsc.asyncGetStatus(sr, 'n_events'))[0]
print('Number of events:', n_events)

Number of events: 1267


In [49]:
neurons

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 1

In [30]:
! /opt/nest/bin/nest-server stop

NEST Server running at http://0.0.0.0:5000 has stopped.


In [31]:
ray.shutdown()